In [3]:
!pip install gensim

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [4]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import gc

In [5]:
#读数据
# train_ad = pd.read_csv('data/train_preliminary/ad.csv')
train_click_log = pd.read_csv('data/train_preliminary/click_log.csv')
# train_user = pd.read_csv('data/train_preliminary/user.csv')

# test_ad = pd.read_csv('data/test/ad.csv')
test_click_log = pd.read_csv('data/test/click_log.csv')

In [6]:
# train_test_ad = train_ad.append(test_ad)
train_test_click_log = train_click_log.append(test_click_log)
# train_test_data = pd.merge(train_test_click_log,train_test_ad,how='outer')
# del train_ad, test_ad, train_click_log, test_click_log
# gc.collect()

In [7]:
train_test_click_log = train_test_click_log.sort_values(by=['user_id', 'time'])
train_test_click_log

,time,user_id,creative_id,click_times
26196513,20,1,821396,1
26196514,20,1,209778,1
26196515,20,1,877468,1
26196517,39,1,1683713,1
26196524,40,1,122032,1
...,...,...,...,...
14648696,75,4000000,3596158,1
14648699,75,4000000,3642395,1
14648674,76,4000000,366858,1
14648681,76,4000000,3333680,1


In [8]:
train_test_id = train_test_click_log[['user_id']].drop_duplicates()
train_test_id

,user_id
26196513,1
12979024,2
29916605,3
1685803,4
18556101,5
...,...
31372714,3999996
14648631,3999997
14648665,3999998
31372771,3999999


In [9]:
train_test_features = train_test_click_log[['user_id', 'creative_id']]
train_test_features

,user_id,creative_id
26196513,1,821396
26196514,1,209778
26196515,1,877468
26196517,1,1683713
26196524,1,122032
...,...,...
14648696,4000000,3596158
14648699,4000000,3642395
14648674,4000000,366858
14648681,4000000,3333680


In [10]:
#造句
def get_sentences(df_, key, name):
    sentences = df_.groupby([key]).apply(lambda x: x[name].tolist()).tolist()
    sentences3 = []
    for i in range(len(sentences)):
        sentences2 = list(map(str, sentences[i]))
        sentences3.append(sentences2)
    return sentences3

sentences = get_sentences(train_test_features, 'user_id', 'creative_id')
len(sentences)

1900000

In [11]:
#w2c
def get_embedding_df(df_, sentences, flag, L, name):
    if flag == True:
        model = Word2Vec(sentences=sentences, size=L, window=10, min_count=2, workers=24, iter=10)
        model.save('output/model/' + name + '_w2c.model')
        print("模型保存完成！！！")
    model = Word2Vec.load('output/model/' + name + '_w2c.model')
    values=set(df_[name].values)
    w2v=[]

    for v in values:
        try:
            a=[int(v)]
            a.extend(model[str(v)])
            w2v.append(a)
        except:
            pass

    out_df=pd.DataFrame(w2v)
    del w2v
    gc.collect()
    out_df.columns = [name]+[name+'_embe_{}'.format(i) for i in range(L)]
#     embedding = pd.merge(df_,out_df,on=name,how='left')
#     del out_df
#     gc.collect()
    return out_df


In [12]:
out_df = get_embedding_df(train_test_features, sentences, True, 32, 'creative_id')
out_df

模型保存完成！！！


/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


,creative_id,creative_id_embe_0,creative_id_embe_1,creative_id_embe_2,creative_id_embe_3,creative_id_embe_4,creative_id_embe_5,creative_id_embe_6,creative_id_embe_7,creative_id_embe_8,...,creative_id_embe_22,creative_id_embe_23,creative_id_embe_24,creative_id_embe_25,creative_id_embe_26,creative_id_embe_27,creative_id_embe_28,creative_id_embe_29,creative_id_embe_30,creative_id_embe_31
0,1,-0.405547,0.725169,0.423469,0.015946,-0.008649,0.077469,-0.010404,0.375016,-0.327518,...,-0.356907,-0.165169,-0.538663,-0.234938,0.825907,0.010920,-0.641028,-0.198432,-0.762730,0.334872
1,4,-0.446732,0.927985,0.526217,0.396676,-0.221806,0.117984,0.039173,0.103609,-0.474610,...,-0.155134,0.002707,0.002844,-0.158829,0.347565,0.020952,-0.983312,-0.704252,-0.359463,-0.147971
2,7,0.087532,0.095536,-0.136538,-0.047536,-0.109545,-0.036275,0.076543,0.023173,-0.146432,...,-0.168074,-0.131901,0.040538,-0.124540,-0.007676,-0.103386,-0.149685,-0.137325,-0.124677,-0.102893
3,10,-0.295333,0.511326,0.373940,0.155660,0.040628,-0.023969,0.073726,0.011495,-0.152683,...,-0.161585,0.079991,-0.252220,-0.216957,0.420880,0.081843,-0.252303,-0.194212,-0.320472,0.274480
4,13,-0.138784,0.324466,0.243483,0.189073,0.284274,0.176511,0.122085,0.048535,-0.010649,...,-0.089465,0.017529,-0.225412,-0.026141,0.210116,-0.298794,-0.401896,-0.276365,-0.028312,0.038636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086581,4445710,-0.064118,0.131664,0.124072,0.059363,0.010769,0.021337,0.021939,0.023574,-0.117052,...,-0.029300,0.021492,-0.065645,-0.050683,0.076507,-0.000035,-0.136281,-0.052887,-0.066511,-0.024349
2086582,4445711,-0.258697,0.148187,0.264473,0.005274,-0.019459,-0.045155,0.040560,0.147630,-0.044312,...,-0.246009,-0.164699,-0.112111,-0.126867,0.070902,-0.229397,-0.150297,-0.190123,-0.059199,0.087896
2086583,4445715,-0.082657,0.229959,0.149897,0.115988,0.000124,0.057486,-0.037380,0.089782,-0.036252,...,-0.088656,0.060387,-0.227509,-0.042947,0.101232,0.033068,-0.107275,-0.152438,-0.148029,0.086677
2086584,4445718,-0.046075,0.064836,0.140005,0.043423,0.035853,0.045022,-0.036449,0.120412,0.026136,...,-0.082540,-0.005498,-0.138193,-0.024613,0.086398,0.032175,-0.078071,-0.081085,-0.093335,0.072537


In [13]:
out_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086586 entries, 0 to 2086585
Data columns (total 33 columns):
 #   Column               Dtype  
---  ------               -----  
 0   creative_id          int64  
 1   creative_id_embe_0   float64
 2   creative_id_embe_1   float64
 3   creative_id_embe_2   float64
 4   creative_id_embe_3   float64
 5   creative_id_embe_4   float64
 6   creative_id_embe_5   float64
 7   creative_id_embe_6   float64
 8   creative_id_embe_7   float64
 9   creative_id_embe_8   float64
 10  creative_id_embe_9   float64
 11  creative_id_embe_10  float64
 12  creative_id_embe_11  float64
 13  creative_id_embe_12  float64
 14  creative_id_embe_13  float64
 15  creative_id_embe_14  float64
 16  creative_id_embe_15  float64
 17  creative_id_embe_16  float64
 18  creative_id_embe_17  float64
 19  creative_id_embe_18  float64
 20  creative_id_embe_19  float64
 21  creative_id_embe_20  float64
 22  creative_id_embe_21  float64
 23  creative_id_embe_22  float64
 24

In [14]:
out_df.to_csv('output/model/creative_id_embedding.csv', index=False)